In [198]:
import sys
sys.path.append('../credentials/') 
from credentials import WeatherCredentials
from twilio.rest import Client
from requests import Request, Session 
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import time 
import os 
import json 


import pandas as pd 
import requests 
from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

## Building URL

In [199]:
query = "Asunción"

credentials = WeatherCredentials()

api_key = credentials.WEATHER_API

In [200]:
api_url = f"http://api.weatherapi.com/v1/forecast.json?key={api_key}&q={query}&days=1&aqi=no&alerts=no"

In [201]:
response = requests.get(api_url).json()

In [202]:
response

{'location': {'name': 'Asunción',
  'region': 'Central',
  'country': 'Paraguay',
  'lat': -25.27,
  'lon': -57.67,
  'tz_id': 'America/Asuncion',
  'localtime_epoch': 1685560509,
  'localtime': '2023-05-31 15:15'},
 'current': {'last_updated_epoch': 1685560500,
  'last_updated': '2023-05-31 15:15',
  'temp_c': 24.0,
  'temp_f': 75.2,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 9.4,
  'wind_kph': 15.1,
  'wind_degree': 50,
  'wind_dir': 'NE',
  'pressure_mb': 1019.0,
  'pressure_in': 30.09,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 69,
  'cloud': 75,
  'feelslike_c': 25.7,
  'feelslike_f': 78.2,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 5.0,
  'gust_mph': 4.0,
  'gust_kph': 6.5},
 'forecast': {'forecastday': [{'date': '2023-05-31',
    'date_epoch': 1685491200,
    'day': {'maxtemp_c': 23.8,
     'maxtemp_f': 74.8,
     'mintemp_c': 15.5,
     'mintemp_f': 59.9,
    

In [203]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [204]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [205]:
response['forecast']['forecastday'][0]['date'] #Date

'2023-05-31'

In [206]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]) #Hour

1

In [207]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text'] #Condition

'Fog'

In [208]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c'] #Temperature

16.6

In [209]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [210]:
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

0

## DataFrame

In [211]:
def get_forecast(response,i):
    date = response['forecast']['forecastday'][0]['date']
    hour = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condition = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temp_c = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    will_it_rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    chance_of_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']

    return date, hour, condition, temp_c, will_it_rain, chance_of_rain
    

In [212]:
data = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour='green'):

    data.append(get_forecast(response,i))



100%|██████████| 24/24 [00:00<?, ?it/s]


In [213]:
##Convert to DF

df = pd.DataFrame(data, columns=['date', 'hour', 'condition', 'temp_c', 'will_it_rain', 'chance_of_rain'])

In [214]:
df.head()

,date,hour,condition,temp_c,will_it_rain,chance_of_rain
0,2023-05-31,0,Fog,16.6,0,0
1,2023-05-31,1,Fog,16.4,0,0
2,2023-05-31,2,Fog,16.2,0,0
3,2023-05-31,3,Fog,16.0,0,0
4,2023-05-31,4,Fog,15.8,0,0


In [215]:
## Looking for cases that have a chance_of_rain greater than 65

df_rain = df[df['chance_of_rain'] > 65]

In [216]:
df_rain

,date,hour,condition,temp_c,will_it_rain,chance_of_rain
14,2023-05-31,14,Patchy rain possible,23.8,0,70


In [217]:
## Message Template

message_template = "Hi, this is your weather report for today. It is currently {temp_c} degrees and {condition}. There is a {chance_of_rain}% chance of rain. Have a nice day!"



## Send SMS using Twilio

In [224]:
time.sleep(3)
account_sid = credentials.TWILIO_ACCOUNT_SID
auth_token = credentials.TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                        body=message_template.format(
                            temp_c=df_rain['temp_c'].values[0],
                            condition=df_rain['condition'].values[0],
                            chance_of_rain=df_rain['chance_of_rain'].values[0]
                        ),
                        from_="+15005550006",
                        to='+595 982 355439'
                    )

print(message.sid)

SM0dbb15cc734b1cb8fcfeac888c7ac69b
